<a href="https://colab.research.google.com/github/MehrabiHasan/South-Park-Analysis/blob/main/Final_SouthPark_Data_Extraction.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

This python code is part of the final project for my database class. This script successfully scrapes [South Park Transcript Website](https://southpark.fandom.com/wiki/South_Park_Archives), and returns dataframes relating to script, episodes and interactions. This code was modified from the medium article [Cartman Chatbot](https://chatbotslife.com/creating-cartmanbot-part-one-61cfe7f40ab7).


In [ ]:
### download neccessary packages
!pip install vaderSentiment
!pip install nltk
!pip install jellyfish
import requests
import re
import pandas as pd
from bs4 import BeautifulSoup, NavigableString, Comment
import string
from vaderSentiment.vaderSentiment import SentimentIntensityAnalyzer
import string
import nltk
nltk.download('punkt')
from nltk.corpus import stopwords
from nltk.tokenize import RegexpTokenizer, word_tokenize
from nltk.stem import WordNetLemmatizer
from nltk.stem.porter import PorterStemmer
from nltk.corpus import stopwords
nltk.download('stopwords')
from jellyfish import soundex

     |████████████████████████████████| 133kB 12.1MB/s 
     |████████████████████████████████| 102kB 9.4MB/s 
[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


In [ ]:
### All the script links from southpark
Season_1_links = ['https://southpark.fandom.com/wiki/Cartman_Gets_an_Anal_Probe/Script','https://southpark.fandom.com/wiki/Weight_Gain_4000/Script','https://southpark.fandom.com/wiki/Volcano/Script','https://southpark.fandom.com/wiki/Big_Gay_Al%27s_Big_Gay_Boat_Ride/Script','https://southpark.fandom.com/wiki/An_Elephant_Makes_Love_to_a_Pig/Script','https://southpark.fandom.com/wiki/Death/Script','https://southpark.fandom.com/wiki/Pinkeye/Script','https://southpark.fandom.com/wiki/Starvin%27_Marvin/Script','https://southpark.fandom.com/wiki/Mr._Hankey,_the_Christmas_Poo/Script','https://southpark.fandom.com/wiki/Damien/Script','https://southpark.fandom.com/wiki/Tom%27s_Rhinoplasty/Script','https://southpark.fandom.com/wiki/Mecha-Streisand/Script','https://southpark.fandom.com/wiki/Cartman%27s_Mom_is_a_Dirty_Slut/Script']
Season_2_links = ['https://southpark.fandom.com/wiki/Terrance_and_Phillip_in_Not_Without_My_Anus/Script','https://southpark.fandom.com/wiki/Cartman%27s_Mom_is_Still_a_Dirty_Slut/Script','https://southpark.fandom.com/wiki/Chickenlover/Script','https://southpark.fandom.com/wiki/Ike%27s_Wee_Wee/Script','https://southpark.fandom.com/wiki/Conjoined_Fetus_Lady/Script','https://southpark.fandom.com/wiki/The_Mexican_Staring_Frog_of_Southern_Sri_Lanka/Script','https://southpark.fandom.com/wiki/City_on_the_Edge_of_Forever_(Flashbacks)/Script','https://southpark.fandom.com/wiki/Summer_Sucks/Script','https://southpark.fandom.com/wiki/Chef%27s_Chocolate_Salty_Balls/Script','https://southpark.fandom.com/wiki/Chickenpox/Script','https://southpark.fandom.com/wiki/Roger_Ebert_Should_Lay_off_the_Fatty_Foods/Script','https://southpark.fandom.com/wiki/Clubhouses/Script','https://southpark.fandom.com/wiki/Cow_Days/Script','https://southpark.fandom.com/wiki/Chef_Aid/Script','https://southpark.fandom.com/wiki/Spookyfish/Script','https://southpark.fandom.com/wiki/Merry_Christmas_Charlie_Manson!/Script','https://southpark.fandom.com/wiki/Gnomes/Script','https://southpark.fandom.com/wiki/Prehistoric_Ice_Man/Script']
Season_3_links = ['https://southpark.fandom.com/wiki/Rainforest_Shmainforest/Script','https://southpark.fandom.com/wiki/Spontaneous_Combustion/Script','https://southpark.fandom.com/wiki/The_Succubus/Script','https://southpark.fandom.com/wiki/Jakovasaurs/Script','https://southpark.fandom.com/wiki/Tweek_vs._Craig/Script','https://southpark.fandom.com/wiki/Sexual_Harassment_Panda/Script','https://southpark.fandom.com/wiki/Cat_Orgy/Script','https://southpark.fandom.com/wiki/Two_Guys_Naked_in_a_Hot_Tub/Script','https://southpark.fandom.com/wiki/Jewbilee/Script','https://southpark.fandom.com/wiki/Korn%27s_Groovy_Pirate_Ghost_Mystery/Script','https://southpark.fandom.com/wiki/Chinpokomon/Script','https://southpark.fandom.com/wiki/Hooked_on_Monkey_Fonics/Script','https://southpark.fandom.com/wiki/Starvin%27_Marvin_in_Space/Script','https://southpark.fandom.com/wiki/The_Red_Badge_of_Gayness/Script','https://southpark.fandom.com/wiki/Mr._Hankey%27s_Christmas_Classics/Script','https://southpark.fandom.com/wiki/Are_You_There_God%3F_It%27s_Me,_Jesus/Script','https://southpark.fandom.com/wiki/World_Wide_Recorder_Concert/Script']
Season_4_links = ['https://southpark.fandom.com/wiki/The_Tooth_Fairy_Tats_2000/Script','https://southpark.fandom.com/wiki/Cartman%27s_Silly_Hate_Crime_2000/Script','https://southpark.fandom.com/wiki/Timmy_2000/Script','https://southpark.fandom.com/wiki/Quintuplets_2000/Script','https://southpark.fandom.com/wiki/Cartman_Joins_NAMBLA/Script','https://southpark.fandom.com/wiki/Cherokee_Hair_Tampons/Script','https://southpark.fandom.com/wiki/Chef_Goes_Nanners/Script','https://southpark.fandom.com/wiki/Something_You_Can_Do_with_Your_Finger/Script','https://southpark.fandom.com/wiki/Do_the_Handicapped_Go_to_Hell%3F/Script','https://southpark.fandom.com/wiki/Probably/Script','https://southpark.fandom.com/wiki/Fourth_Grade/Script','https://southpark.fandom.com/wiki/Trapper_Keeper/Script','https://southpark.fandom.com/wiki/Helen_Keller!_The_Musical/Script','https://southpark.fandom.com/wiki/Pip/Script','https://southpark.fandom.com/wiki/Fat_Camp/Script','https://southpark.fandom.com/wiki/The_Wacky_Molestation_Adventure/Script','https://southpark.fandom.com/wiki/A_Very_Crappy_Christmas/Script']
Season_5_links = ['https://southpark.fandom.com/wiki/It_Hits_the_Fan/Script','https://southpark.fandom.com/wiki/Cripple_Fight/Script','https://southpark.fandom.com/wiki/Super_Best_Friends/Script','https://southpark.fandom.com/wiki/Scott_Tenorman_Must_Die/Script','https://southpark.fandom.com/wiki/Terrance_and_Phillip:_Behind_the_Blow/Script','https://southpark.fandom.com/wiki/Cartmanland/Script','https://southpark.fandom.com/wiki/Proper_Condom_Use/Script','https://southpark.fandom.com/wiki/Towelie/Script','https://southpark.fandom.com/wiki/Osama_bin_Laden_Has_Farty_Pants/Script','https://southpark.fandom.com/wiki/How_to_Eat_with_Your_Butt/Script','https://southpark.fandom.com/wiki/The_Entity/Script','https://southpark.fandom.com/wiki/Here_Comes_the_Neighborhood/Script','https://southpark.fandom.com/wiki/Kenny_Dies/Script','https://southpark.fandom.com/wiki/Butters%27_Very_Own_Episode/Script']
Season_6_links = ['https://southpark.fandom.com/wiki/Jared_Has_Aides/Script','https://southpark.fandom.com/wiki/Asspen/Script','https://southpark.fandom.com/wiki/Freak_Strike/Script','https://southpark.fandom.com/wiki/Fun_with_Veal/Script','https://southpark.fandom.com/wiki/The_New_Terrance_and_Phillip_Movie_Trailer/Script','https://southpark.fandom.com/wiki/Professor_Chaos/Script','https://southpark.fandom.com/wiki/The_Simpsons_Already_Did_It/Script','https://southpark.fandom.com/wiki/Red_Hot_Catholic_Love/Script','https://southpark.fandom.com/wiki/Free_Hat/Script','https://southpark.fandom.com/wiki/Bebe%27s_Boobs_Destroy_Society/Script','https://southpark.fandom.com/wiki/Child_Abduction_is_Not_Funny/Script','https://southpark.fandom.com/wiki/A_Ladder_to_Heaven/Script','https://southpark.fandom.com/wiki/The_Return_of_the_Fellowship_of_the_Ring_to_the_Two_Towers/Script','https://southpark.fandom.com/wiki/The_Death_Camp_of_Tolerance/Script','https://southpark.fandom.com/wiki/The_Biggest_Douche_in_the_Universe/Script','https://southpark.fandom.com/wiki/My_Future_Self_n%27_Me/Script','https://southpark.fandom.com/wiki/Red_Sleigh_Down/Script']
Season_7_links = ['https://southpark.fandom.com/wiki/Cancelled/Script','https://southpark.fandom.com/wiki/Krazy_Kripples/Script','https://southpark.fandom.com/wiki/Toilet_Paper/Script','https://southpark.fandom.com/wiki/I%27m_a_Little_Bit_Country/Script','https://southpark.fandom.com/wiki/Fat_Butt_and_Pancake_Head/Script','https://southpark.fandom.com/wiki/Lil%27_Crime_Stoppers/Script','https://southpark.fandom.com/wiki/Red_Man%27s_Greed/Script','https://southpark.fandom.com/wiki/South_Park_is_Gay!/Script','https://southpark.fandom.com/wiki/Christian_Rock_Hard/Script','https://southpark.fandom.com/wiki/Grey_Dawn/Script','https://southpark.fandom.com/wiki/Casa_Bonita/Script','https://southpark.fandom.com/wiki/All_About_Mormons/Script','https://southpark.fandom.com/wiki/Butt_Out/Script','https://southpark.fandom.com/wiki/Raisins/Script','https://southpark.fandom.com/wiki/It%27s_Christmas_in_Canada/Script']
Season_8_links = ['https://southpark.fandom.com/wiki/Good_Times_with_Weapons/Script','https://southpark.fandom.com/wiki/Up_the_Down_Steroid/Script','https://southpark.fandom.com/wiki/The_Passion_of_the_Jew/Script','https://southpark.fandom.com/wiki/You_Got_F%27d_in_the_A/Script','https://southpark.fandom.com/wiki/AWESOM-O/Script','https://southpark.fandom.com/wiki/The_Jeffersons/Script','https://southpark.fandom.com/wiki/Goobacks/Script','https://southpark.fandom.com/wiki/Douche_and_Turd/Script','https://southpark.fandom.com/wiki/Something_Wall-Mart_This_Way_Comes/Script','https://southpark.fandom.com/wiki/Pre-School/Script','https://southpark.fandom.com/wiki/Quest_for_Ratings/Script','https://southpark.fandom.com/wiki/Stupid_Spoiled_Whore_Video_Playset/Script','https://southpark.fandom.com/wiki/Cartman%27s_Incredible_Gift/Script','https://southpark.fandom.com/wiki/Woodland_Critter_Christmas/Script']
Season_9_links = ['https://southpark.fandom.com/wiki/Mr._Garrison%27s_Fancy_New_Vagina/Script','https://southpark.fandom.com/wiki/Die_Hippie,_Die/Script','https://southpark.fandom.com/wiki/Wing/Script','https://southpark.fandom.com/wiki/Best_Friends_Forever/Script','https://southpark.fandom.com/wiki/The_Losing_Edge/Script','https://southpark.fandom.com/wiki/The_Death_of_Eric_Cartman/Script','https://southpark.fandom.com/wiki/Erection_Day/Script','https://southpark.fandom.com/wiki/Two_Days_Before_the_Day_After_Tomorrow/Script','https://southpark.fandom.com/wiki/Marjorine/Script','https://southpark.fandom.com/wiki/Follow_That_Egg!/Script','https://southpark.fandom.com/wiki/Ginger_Kids/Script','https://southpark.fandom.com/wiki/Trapped_in_the_Closet/Script','https://southpark.fandom.com/wiki/Free_Willzyx/Script','https://southpark.fandom.com/wiki/Bloody_Mary/Script']
Season_10_links = ['https://southpark.fandom.com/wiki/The_Return_of_Chef/Script','https://southpark.fandom.com/wiki/Smug_Alert!/Script','https://southpark.fandom.com/wiki/Cartoon_Wars_Part_I/Script','https://southpark.fandom.com/wiki/Cartoon_Wars_Part_II/Script','https://southpark.fandom.com/wiki/A_Million_Little_Fibers/Script','https://southpark.fandom.com/wiki/ManBearPig/Script','https://southpark.fandom.com/wiki/Tsst/Script','https://southpark.fandom.com/wiki/Make_Love,_Not_Warcraft/Script','https://southpark.fandom.com/wiki/Mystery_of_the_Urinal_Deuce/Script','https://southpark.fandom.com/wiki/Miss_Teacher_Bangs_a_Boy/Script','https://southpark.fandom.com/wiki/Hell_on_Earth_2006/Script','https://southpark.fandom.com/wiki/Go_God_Go/Script','https://southpark.fandom.com/wiki/Go_God_Go_XII/Script','https://southpark.fandom.com/wiki/Stanley%27s_Cup/Script']
Season_11_links = ['https://southpark.fandom.com/wiki/With_Apologies_to_Jesse_Jackson/Script','https://southpark.fandom.com/wiki/Cartman_Sucks/Script','https://southpark.fandom.com/wiki/Lice_Capades/Script','https://southpark.fandom.com/wiki/The_Snuke/Script','https://southpark.fandom.com/wiki/Fantastic_Easter_Special/Script','https://southpark.fandom.com/wiki/D-Yikes!/Script','https://southpark.fandom.com/wiki/Night_of_the_Living_Homeless/Script','https://southpark.fandom.com/wiki/Le_Petit_Tourette/Script','https://southpark.fandom.com/wiki/More_Crap/Script','https://southpark.fandom.com/wiki/Imaginationland/Script','https://southpark.fandom.com/wiki/Imaginationland,_Episode_II/Script','https://southpark.fandom.com/wiki/Imaginationland,_Episode_III/Script','https://southpark.fandom.com/wiki/Guitar_Queer-O/Script','https://southpark.fandom.com/wiki/The_List/Script']
Season_12_links = ['https://southpark.fandom.com/wiki/Tonsil_Trouble/Script','https://southpark.fandom.com/wiki/Britney%27s_New_Look/Script','https://southpark.fandom.com/wiki/Major_Boobage/Script','https://southpark.fandom.com/wiki/Canada_on_Strike/Script','https://southpark.fandom.com/wiki/Eek,_A_Penis!/Script','https://southpark.fandom.com/wiki/Over_Logging/Script','https://southpark.fandom.com/wiki/Super_Fun_Time/Script','https://southpark.fandom.com/wiki/The_China_Probrem/Script','https://southpark.fandom.com/wiki/Breast_Cancer_Show_Ever/Script','https://southpark.fandom.com/wiki/Pandemic/Script','https://southpark.fandom.com/wiki/Pandemic_2:_The_Startling/Script','https://southpark.fandom.com/wiki/About_Last_Night.../Script','https://southpark.fandom.com/wiki/Elementary_School_Musical/Script','https://southpark.fandom.com/wiki/The_Ungroundable/Script']
Season_13_links = ['https://southpark.fandom.com/wiki/The_Ring/Script','https://southpark.fandom.com/wiki/The_Coon/Script','https://southpark.fandom.com/wiki/Margaritaville/Script','https://southpark.fandom.com/wiki/Eat,_Pray,_Queef/Script','https://southpark.fandom.com/wiki/Fishsticks/Script','https://southpark.fandom.com/wiki/Pinewood_Derby/Script','https://southpark.fandom.com/wiki/Fatbeard/Script','https://southpark.fandom.com/wiki/Dead_Celebrities/Script','https://southpark.fandom.com/wiki/Butters%27_Bottom_Bitch/Script','https://southpark.fandom.com/wiki/W.T.F./Script','https://southpark.fandom.com/wiki/Whale_Whores/Script','https://southpark.fandom.com/wiki/The_F_Word/Script','https://southpark.fandom.com/wiki/Dances_with_Smurfs/Script','https://southpark.fandom.com/wiki/Pee/Script']
Season_14_links = ['https://southpark.fandom.com/wiki/Sexual_Healing/Script','https://southpark.fandom.com/wiki/The_Tale_of_Scrotie_McBoogerballs/Script','https://southpark.fandom.com/wiki/Medicinal_Fried_Chicken/Script','https://southpark.fandom.com/wiki/You_Have_0_Friends/Script','https://southpark.fandom.com/wiki/200/Script','https://southpark.fandom.com/wiki/201/Script','https://southpark.fandom.com/wiki/Crippled_Summer/Script','https://southpark.fandom.com/wiki/Poor_and_Stupid/Script','https://southpark.fandom.com/wiki/It%27s_a_Jersey_Thing/Script','https://southpark.fandom.com/wiki/Insheeption/Script','https://southpark.fandom.com/wiki/Coon_2:_Hindsight/Script','https://southpark.fandom.com/wiki/Mysterion_Rises/Script','https://southpark.fandom.com/wiki/Coon_vs._Coon_%26_Friends/Script','https://southpark.fandom.com/wiki/Cr%C3%A8me_Fraiche/Script']
Season_15_links = ['https://southpark.fandom.com/wiki/HUMANCENTiPAD/Script','https://southpark.fandom.com/wiki/Funnybot/Script','https://southpark.fandom.com/wiki/Royal_Pudding/Script','https://southpark.fandom.com/wiki/T.M.I./Script','https://southpark.fandom.com/wiki/Crack_Baby_Athletic_Association/Script','https://southpark.fandom.com/wiki/City_Sushi/Script','https://southpark.fandom.com/wiki/You%27re_Getting_Old/Script','https://southpark.fandom.com/wiki/Ass_Burgers/Script','https://southpark.fandom.com/wiki/The_Last_of_the_Meheecans/Script','https://southpark.fandom.com/wiki/Bass_to_Mouth/Script','https://southpark.fandom.com/wiki/Broadway_Bro_Down/Script','https://southpark.fandom.com/wiki/1%25/Script','https://southpark.fandom.com/wiki/A_History_Channel_Thanksgiving/Script','https://southpark.fandom.com/wiki/The_Poor_Kid/Script']
Season_16_links = ['https://southpark.fandom.com/wiki/Reverse_Cowgirl/Script','https://southpark.fandom.com/wiki/Cash_For_Gold/Script','https://southpark.fandom.com/wiki/Faith_Hilling/Script','https://southpark.fandom.com/wiki/Jewpacabra/Script','https://southpark.fandom.com/wiki/Butterballs/Script','https://southpark.fandom.com/wiki/I_Should_Have_Never_Gone_Ziplining/Script','https://southpark.fandom.com/wiki/Cartman_Finds_Love/Script','https://southpark.fandom.com/wiki/Sarcastaball/Script','https://southpark.fandom.com/wiki/Raising_the_Bar/Script','https://southpark.fandom.com/wiki/Insecurity/Script','https://southpark.fandom.com/wiki/Going_Native/Script','https://southpark.fandom.com/wiki/A_Nightmare_on_Face_Time/Script','https://southpark.fandom.com/wiki/A_Scause_For_Applause/Script','https://southpark.fandom.com/wiki/Obama_Wins!/Script']
Season_17_links = ['https://southpark.fandom.com/wiki/Let_Go,_Let_Gov/Script','https://southpark.fandom.com/wiki/Informative_Murder_Porn/Script','https://southpark.fandom.com/wiki/World_War_Zimmerman/Script','https://southpark.fandom.com/wiki/Goth_Kids_3:_Dawn_of_the_Posers/Script','https://southpark.fandom.com/wiki/Taming_Strange/Script','https://southpark.fandom.com/wiki/Ginger_Cow/Script','https://southpark.fandom.com/wiki/Black_Friday/Script','https://southpark.fandom.com/wiki/A_Song_of_Ass_and_Fire/Script','https://southpark.fandom.com/wiki/Titties_and_Dragons/Script','https://southpark.fandom.com/wiki/The_Hobbit/Script']
Season_18_links = ['https://southpark.fandom.com/wiki/Go_Fund_Yourself/Script','https://southpark.fandom.com/wiki/Gluten_Free_Ebola/Script','https://southpark.fandom.com/wiki/The_Cissy/Script','https://southpark.fandom.com/wiki/Handicar/Script','https://southpark.fandom.com/wiki/The_Magic_Bush/Script','https://southpark.fandom.com/wiki/Freemium_Isn%27t_Free/Script','https://southpark.fandom.com/wiki/Grounded_Vindaloop/Script','https://southpark.fandom.com/wiki/Cock_Magic/Script','https://southpark.fandom.com/wiki/REHASH/Script','https://southpark.fandom.com/wiki/HappyHolograms/Script']
Season_19_links = ['https://southpark.fandom.com/wiki/Stunning_and_Brave/Script','https://southpark.fandom.com/wiki/Where_My_Country_Gone%3F/Script','https://southpark.fandom.com/wiki/The_City_Part_of_Town/Script','https://southpark.fandom.com/wiki/You%27re_Not_Yelping/Script','https://southpark.fandom.com/wiki/Safe_Space/Script','https://southpark.fandom.com/wiki/Tweek_x_Craig/Script','https://southpark.fandom.com/wiki/Naughty_Ninjas/Script','https://southpark.fandom.com/wiki/Sponsored_Content/Script','https://southpark.fandom.com/wiki/Truth_and_Advertising/Script','https://southpark.fandom.com/wiki/PC_Principal_Final_Justice/Script']
Season_20_links = ['https://southpark.fandom.com/wiki/Member_Berries/Script','https://southpark.fandom.com/wiki/Skank_Hunt/Script','https://southpark.fandom.com/wiki/The_Damned/Script','https://southpark.fandom.com/wiki/Wieners_Out/Script','https://southpark.fandom.com/wiki/Douche_and_a_Danish/Script','https://southpark.fandom.com/wiki/Fort_Collins/Script','https://southpark.fandom.com/wiki/Oh,_Jeez/Script','https://southpark.fandom.com/wiki/Members_Only/Script','https://southpark.fandom.com/wiki/Not_Funny/Script','https://southpark.fandom.com/wiki/The_End_of_Serialization_as_We_Know_It/Script']
Season_21_links = ['https://southpark.fandom.com/wiki/White_People_Renovating_Houses/Script','https://southpark.fandom.com/wiki/Put_It_Down/Script','https://southpark.fandom.com/wiki/Holiday_Special/Script','https://southpark.fandom.com/wiki/Franchise_Prequel/Script','https://southpark.fandom.com/wiki/Hummels_%26_Heroin/Script','https://southpark.fandom.com/wiki/Sons_A_Witches/Script','https://southpark.fandom.com/wiki/Doubling_Down/Script','https://southpark.fandom.com/wiki/Moss_Piglets/Script','https://southpark.fandom.com/wiki/SUPER_HARD_PCness/Script','https://southpark.fandom.com/wiki/Splatty_Tomato/Script']
Season_22_links = ['https://southpark.fandom.com/wiki/Dead_Kids/Script','https://southpark.fandom.com/wiki/A_Boy_And_A_Priest/Script','https://southpark.fandom.com/wiki/The_Problem_with_a_Poo/Script','https://southpark.fandom.com/wiki/Tegridy_Farms/Script','https://southpark.fandom.com/wiki/The_Scoots/Script','https://southpark.fandom.com/wiki/Time_To_Get_Cereal/Script','https://southpark.fandom.com/wiki/Nobody_Got_Cereal%3F/Script','https://southpark.fandom.com/wiki/Buddha_Box/Script','https://southpark.fandom.com/wiki/Unfulfilled/Script','https://southpark.fandom.com/wiki/Bike_Parade/Script']
Season_23_links = ['https://southpark.fandom.com/wiki/Mexican_Joker/Script','https://southpark.fandom.com/wiki/Band_in_China/Script','https://southpark.fandom.com/wiki/Shots/Script','https://southpark.fandom.com/wiki/Let_Them_Eat_Goo/Script','https://southpark.fandom.com/wiki/Tegridy_Farms_Halloween_Special/Script','https://southpark.fandom.com/wiki/Season_Finale/Script','https://southpark.fandom.com/wiki/Board_Girls/Script','https://southpark.fandom.com/wiki/Turd_Burglars/Script','https://southpark.fandom.com/wiki/Basic_Cable/Script','https://southpark.fandom.com/wiki/Christmas_Snow/Script']
All_Season_links = [Season_1_links,Season_2_links,Season_3_links,Season_4_links,Season_5_links,Season_6_links,Season_7_links,Season_8_links,Season_9_links,Season_10_links,Season_11_links,Season_12_links,Season_13_links,Season_14_links,Season_15_links,Season_16_links,Season_17_links,Season_18_links,Season_19_links,Season_20_links,Season_21_links,Season_22_links,Season_23_links]

In [ ]:
### Helper Functions

### Cleaning the lines
def line_cleaning(line):
  remove = string.punctuation
  remove = remove.replace('[',"")
  remove = remove.replace("]","")
  remove = remove.replace("'","")
  remove = remove.replace('''"''',"")
  remove = remove.replace(".","")
  remove = remove.replace("...","")
  remove = remove.replace("!","")
  remove = remove.replace("?","")
  remove = remove.replace("-","")
  remove = remove.replace(",","")
  pattern = r"[{}]".format(remove)
  line = line.translate(str.maketrans("","",remove))


  line = re.sub("([\(\[]).*?([\)\]])", "",line)

  line = line.strip()

  return line

### Creating Events Column
def format_1_ev(m):
  soup = m
  lines = [x.nextSibling.nextSibling for x in soup.find_all('td', style="border-bottom: 1px solid #BBB; border-right: 1px solid #CCC;") if x.text.replace('\n','') != ""]
  events = []
  scene_num = 0
  stri = ""
  p = soup.find_all('td',attrs = {'style' : 'border-bottom: 1px solid #BBB; background-color:#6dc56d; padding-left:5px'})
  z = soup.find('td',attrs = {'style' : 'border-bottom: 1px solid #BBB; background-color:#6dc56d; padding-left:5px'})
  temp = []
  t = soup.find_all('td')
  for x in t:
    if x in p :
      temp.append('Event')
    if x in lines:
      temp.append('Line')
  for e in temp:
    if e == 'Line':
      events.append('Scene '+ str(scene_num))
    else:
      scene_num += 1
  return events

def format_2_ev(main):
  soup = main
  event_list = []
  p = soup.findAll('td',attrs={'bgcolor':'#6dc56d'})
  for i in p:
    event_list.append(i)


  c = []
  l = []
  temp = []
  for x in soup.findAll('th'):
            try:
                t = x.text.replace('\n','')
                c.append(t)
                v = x.nextSibling.nextSibling
                temp.append(v)
                if (v not in p):
                  l.append(v.text.replace('\n',''))

            except:
              continue

  e = []
  scene_num = 0   
  for i in temp:
    t = 'Scene ' + str(scene_num)
    if i in p:
      scene_num += 1
    else:
      e.append(t)

  while '' in c:
    c.remove('')

  return e,l,c

In [ ]:
### Go Through And Make ScenesDataframe - [Season,Episode,Character,Line,Scene]
def Get_Season(Season,Snum):
  DF = pd.DataFrame(columns=['Season','Episode','Character','Line','Scene'])
  counter = 1
  for i in range(len(Season)):
    temp = main(Season[i],Snum,i+1)
    counter += 1
    DF = pd.concat([DF,temp],axis=0)
  return DF

def main(Filename,Season,Episode):
  characters = []
  lines = []
  events = []
  page = requests.get(Filename)
  soup = BeautifulSoup(page.content, 'html.parser')
  # Checking if the url fits the 1st format
  if soup.findAll('td', style="border-bottom: 1px solid #BBB; border-right: 1px solid #CCC;") != []:

    for i in format_1_ev(soup):
      events.append(i)
    for x in soup.find_all('td', style="border-bottom: 1px solid #BBB; border-right: 1px solid #CCC;"):
    
# Checking if there is a character speaking
        if x.text.replace('\n', '') != "":
# If a character is speaking, add characters and lines to the list
            t = x.text.replace('\n', '')
            v = x.nextSibling.nextSibling.text.replace('\n', '')
            characters.append(t)
            lines.append(v)
        else:
          continue 
  else:
    e,l,c = format_2_ev(soup)
    for x,y,z in zip(e,l,c):
      events.append(x)
      lines.append(y)
      characters.append(z)
  if Season == 12 and Episode == 12:
    event_new = events[:-1]
    events = event_new


  for i in range(len(lines)):
    lines[i] = line_cleaning(lines[i])
    characters[i] = re.sub(":", "", characters[i])
    df = pd.DataFrame({'Season':Season,'Episode':Episode,'Character':characters, 'Line':lines, 'Scene':events})
  return df



In [ ]:
#Season 1
S1 = Get_Season(Season_1_links,1)


In [ ]:
###Season 2 
S2 = Get_Season(Season_2_links,2)


In [ ]:
#Season 3
S3 = Get_Season(Season_3_links,3)


In [ ]:
S4 = Get_Season(Season_4_links,4)


In [ ]:
S5 = Get_Season(Season_5_links,5)


In [ ]:
S6 = Get_Season(Season_6_links,6)


In [ ]:
S7 = Get_Season(Season_7_links,7)


In [ ]:
S8 = Get_Season(Season_8_links,8)


In [ ]:
S9 = Get_Season(Season_9_links,9)


In [ ]:
S10 = Get_Season(Season_10_links,10)


,Season,Episode,Character,Line,Scene
213,10,14,Judge,Stan Marsh coaches a Pee Wee hockey team. But ...,Scene 19
214,10,14,Av 1,"All right! Give 'em hell, boys!",Scene 19
215,10,14,The Avs,All right! Yeah!,Scene 19
216,10,14,Announcer,It's a tie game here in Colorado as we get set...,Scene 20
217,10,14,Denver Fans,OHHHHHHHHH!,Scene 21
218,10,14,Stan,Jesus Christ! Stop! Stop this!,Scene 21
219,10,14,Man,Let them play!,Scene 21
220,10,14,Fans,Let them play! Let them play!,Scene 21
221,10,14,# 15,EEEK-ugh,Scene 21
222,10,14,Stan,"Hey, can they do that??",Scene 22


In [ ]:
S11 = Get_Season(Season_11_links,11)


In [ ]:
S12 = Get_Season(Season_12_links,12)


In [ ]:
S13 = Get_Season(Season_13_links,13)


In [ ]:
S14 = Get_Season(Season_14_links,14)


In [ ]:
S15 = Get_Season(Season_15_links,15)


In [ ]:
S16 = Get_Season(Season_16_links,16)


In [ ]:
S17 = Get_Season(Season_17_links,17)


In [ ]:
S18 = Get_Season(Season_18_links,18)


In [ ]:
S19 = Get_Season(Season_19_links,19)


In [ ]:
S20 = Get_Season(Season_20_links,20)


In [ ]:
S21 = Get_Season(Season_21_links,21)


In [ ]:
S22 = Get_Season(Season_22_links,22)


In [ ]:
S23 = Get_Season(Season_23_links,23)


In [ ]:
#### Combine into one 
Script_DF = pd.DataFrame(columns=['Season','Episode','Character','Line','Scene'])
Script_DF = pd.concat([Script_DF,S1],axis=0).reset_index(drop=True)
Script_DF = pd.concat([Script_DF,S2],axis=0).reset_index(drop=True)
Script_DF = pd.concat([Script_DF,S3],axis=0).reset_index(drop=True)
Script_DF = pd.concat([Script_DF,S4],axis=0).reset_index(drop=True)
Script_DF = pd.concat([Script_DF,S5],axis=0).reset_index(drop=True)
Script_DF = pd.concat([Script_DF,S6],axis=0).reset_index(drop=True)
Script_DF = pd.concat([Script_DF,S7],axis=0).reset_index(drop=True)
Script_DF = pd.concat([Script_DF,S8],axis=0).reset_index(drop=True)
Script_DF = pd.concat([Script_DF,S9],axis=0).reset_index(drop=True)
Script_DF = pd.concat([Script_DF,S10],axis=0).reset_index(drop=True)
Script_DF = pd.concat([Script_DF,S11],axis=0).reset_index(drop=True)
Script_DF = pd.concat([Script_DF,S12],axis=0).reset_index(drop=True)
Script_DF = pd.concat([Script_DF,S13],axis=0).reset_index(drop=True)
Script_DF = pd.concat([Script_DF,S14],axis=0).reset_index(drop=True)
Script_DF = pd.concat([Script_DF,S15],axis=0).reset_index(drop=True)
Script_DF = pd.concat([Script_DF,S16],axis=0).reset_index(drop=True)
Script_DF = pd.concat([Script_DF,S17],axis=0).reset_index(drop=True)
Script_DF = pd.concat([Script_DF,S18],axis=0).reset_index(drop=True)
Script_DF = pd.concat([Script_DF,S19],axis=0).reset_index(drop=True)
Script_DF = pd.concat([Script_DF,S20],axis=0).reset_index(drop=True)
Script_DF = pd.concat([Script_DF,S21],axis=0).reset_index(drop=True)
Script_DF = pd.concat([Script_DF,S22],axis=0).reset_index(drop=True)
Script_DF = pd.concat([Script_DF,S23],axis=0).reset_index(drop=True)
Script_DF

,Season,Episode,Character,Line,Scene
0,1,1,The Boys,"School days, school days, teacher's golden ru...",Scene 1
1,1,1,Kyle Broflovski,"Ah, damn it! My little brother's trying to fol...",Scene 1
2,1,1,Ike Broflovski,Ee gko zeeponanner.,Scene 1
3,1,1,Kyle,"Ike, you can't come to school with me.",Scene 1
4,1,1,Eric Cartman,"Yeah, go home you little dildo!",Scene 1
...,...,...,...,...,...
83257,23,10,Santa,It's more than okay. Tegridy cocaine is smooth...,Scene 24
83258,23,10,Jesus,"I'll bet when coke becomes legal soon, everyon...",Scene 24
83259,23,10,Stephen,"Well, only one thing left to do, gang. Let's g...",Scene 24
83260,23,10,All,Yeah!,Scene 24


In [ ]:
Script_DF = pd.read_csv('Cleaned Names.csv')
Script_DF.head(5)
temp = Script_DF.groupby(['Season','Episode']).agg(['mean','std','var'])
temp.to_csv('sentiment_grouping.csv')

In [ ]:
### Sentiment Analysis 
def sentiment_analysis(sentence):
    sid_obj = SentimentIntensityAnalyzer()
    sentiment_dictionary = sid_obj.polarity_scores(sentence)
    return sentiment_dictionary['compound']

def Clean_Comments(Sentence):
    word_list = []
    punc = '''!()[]{};:'"\, <>./?@#$%^&*_~'''
    Sentence = Sentence.lower()
    Sentence = word_tokenize(Sentence)
    stop_words = set(stopwords.words('english')) 
    for words in Sentence:
        if words not in punc and words not in stop_words:
            word_list.append(words)
    Sentence[:] = [' '.join(word_list[:])]
    Sentiment = sentiment_analysis(Sentence)
    return Sentiment

sent_list = []
counter = 1
for i in Script_DF['Line']:
  sent = Clean_Comments(i)
  print(f'This Calculation is {(counter/len(Script_DF))*100}% done')
  counter += 1
  sent_list.append(sent)
Script_DF['Sentiment'] = sent_list

NameError: ignored

In [ ]:
#### Print Finished Script DF 
Script_DF.to_csv('All_Season_Script.csv')

In [ ]:
Script_DF = pd.read_csv('All_Season_Script.csv')

In [ ]:
### Methods to Clean Character Names
def name_matcher (datalist,name):
  regx= re.compile(r'(?m)^\s*([A-Z\s]*?(?:('+str(name)+')).*)$')
  name_list = []
  newlist = filter(regx.match, datalist) # Read Note
  for i in newlist:
    name_list.append(i)
  return name_list

def Clean_Character_Names(Season,Episode,old,new,DF):
  temp = DF.copy()
  s = int(Season)
  e = int(Episode)
  t = DF['Character'].loc[(DF['Season'] == s)& (DF['Episode'] == e) & (DF['Character'] == old)]
  for row,y in t.iteritems():
     temp.loc[row,'Character'] = new
  return temp


dataList = df['Character'].apply(str).unique().tolist()
name_matcher(dataList,'Dr. Mephesto')

['Dr. Mephesto']

In [ ]:
#### Clean Character Names for entire dataframe
DF = Script_DF
DF['Character'] = DF['Character'].str.replace('Kyle Broflovski','Kyle')
DF['Character'] = DF['Character'].str.replace('Eric Cartman','Eric')
DF['Character'] = DF['Character'].str.replace('Stan Marsh','Stan')
DF['Character'] = DF['Character'].str.replace('Kenny McCormick','Kenny')
DF['Character'] = DF['Character'].str.replace('Ike Broflovski','Ike')
DF['Character'] = DF['Character'].str.replace('Ms. Veronica Crabtree','Ms. Crabtree')
DF['Character'] = DF['Character'].str.replace('Farmer Carl Denkins','Farmer Carl')
DF['Character'] = DF['Character'].str.replace('Mr. Herbert Garrison','Mr. Garrison')
DF['Character'] = DF['Character'].str.replace('Wendy Testaburger','Wendy')
DF['Character'] = DF['Character'].str.replace('Liane Cartman','Liane')
DF['Character'] = DF['Character'].str.replace('Bebe Stevens','Bebe')
DF['Character'] = DF['Character'].str.replace('Randy Marsh','Randy')
DF['Character'] = DF['Character'].str.replace('Jimbo Kern','Jimbo')
DF['Character'] = DF['Character'].str.replace('Sheila Broflovski','Sheila')
DF['Character'] = DF['Character'].str.replace('Shelley','Shelly')
DF['Character'] = DF['Character'].str.replace('Shelly Marsh','Shelly')
DF['Character'] = DF['Character'].str.replace('Butters Stotch','Butters')
DF['Character'] = DF['Character'].str.replace('Butters\tOh. Okay then.','Butters')
DF['Character'] = DF['Character'].str.replace('RandyWho are they? [five new guys are in the recording booth]','Randy')
DF['Character'] = DF['Character'].str.replace('Randy (singing)','Randy')
DF['Character'] = DF['Character'].str.replace('Sharon Marsh','Sharon')
DF['Character'] = DF['Character'].str.replace('Gerald Broflovski','Gerald')
DF['Character'] = DF['Character'].str.replace('Jimmy Valmer','Jimmy')
DF['Character'] = DF['Character'].str.replace('Timmy Burch','Timmy')
DF['Character'] = DF['Character'].str.replace('Clyde Donovan','Clyde')
DF['Character'] = DF['Character'].str.replace('Craig Tucker','Craig')
DF['Character'] = DF['Character'].str.replace('Token Black','Token')
DF['Character'] = DF['Character'].str.replace("Token's office, nicely appointed. some loud pounding is heard on the door ]",'Token')
DF['Character'] = DF['Character'].str.replace("Tweek Tweak",'Tweek')
DF['Character'] = DF['Character'].str.replace("Priest Maxi",'Father Maxi')
DF['Character'] = DF['Character'].str.replace("Stuart",'Stuart McCormick')
DF['Character'] = DF['Character'].str.replace('Stuart McCormick McCormick','Stuart McCormick')
DF['Character'] = DF['Character'].str.replace('Richard Tweek','Richard Tweak')
DF['Character'] = DF['Character'].str.replace('Eric','Cartman')
DF['Character'] = DF['Character'].str.replace('Visitor Carl','Farmer Carl')
DF['Character'] = DF['Character'].str.replace('Ned Gerblansky','Ned')
DF['Character'] = DF['Character'].str.replace('Dr. Alphonse Mephesto','Mephesto')
DF['Character'] = DF['Character'].str.replace('Uncle Jimbo','Jimbo')
DF['Character'] = DF['Character'].str.replace('Trainee Barbrady','Officer Barbrady')
DF['Character'] = DF['Character'].str.replace('Falsetto Bob Saget','Bob Saget')
DF['Character'] = DF['Character'].str.replace('Scott Malkinson the Dick','Scott Malkinson')
DF['Character'] = DF['Character'].str.replace('Saddam Hussein','Suddam')
DF['Character'] = DF['Character'].str.replace('Mephesto Tryout','Mephesto')
DF['Character'] = DF['Character'].str.replace('Eric Roberts as Kevin','Eric Roberts')
DF['Character'] = DF['Character'].str.replace('Chef (recording)','Chef')
DF['Character'] = DF['Character'].str.replace('Chef (falsetto)','Chef')
DF['Character'] = DF['Character'].str.replace('Dr. Mephesto','Mephesto')
DF['Character'] = DF['Character'].str.replace('Fr. Maxi','Father Maxi')
DF['Character'] = DF['Character'].str.replace('Officer Barbrady','Barbrady')
DF['Character'] = DF['Character'].str.replace('Russell Crowe (off-screen)','Russell Crowe')
DF['Character'] = DF['Character'].str.replace("Tuong Lu Kim",'Mr. Kim')
DF['Character'] = DF['Character'].str.replace("Father Maxi Barnes",'Father Maxi')
DF['Character'] = DF['Character'].str.replace("Mrs. Garrison",'Mr. Garrison')
DF['Character'] = DF['Character'].str.replace("Mr. Garrison",'Mr./Mrs. Garrison')
DF['Character'] = DF['Character'].str.replace("Professor Chaos",'Butters')

In [ ]:
new = Clean_Character_Names(1,5,'Terrance','Terrance Mephesto',DF)
new2 = Clean_Character_Names(1,5,'Richard','Richard Tweak',new)
new3 = Clean_Character_Names(22,6,'Marvin','Grandpa',new2)
new4 = Clean_Character_Names(22,7,'Marvin','Grandpa',new3)

In [ ]:
Cleaned_DF = new4
Cleaned_DF.to_csv("Cleaned Names.csv")

In [ ]:
###Create Interaction Table - Directed
import itertools
Cleaned_DF = new4
Group_DF = pd.DataFrame(columns=['Season','Episode','Scene','Characters'])
grouped_df = Cleaned_DF.groupby(['Season','Episode','Scene'])
t = df.groupby(['Season','Episode','Scene'])['Character'].agg(lambda x: ','.join(set(x))).reset_index()
s = []
e = []
sc = []
ch = []
for name,group in t.iterrows():
  if len(group[3].split(',')) > 1:
    s.append(group[0])
    e.append(group[1])
    sc.append(group[2])
    ch.append(group[3])
Group_DF['Season'] = s
Group_DF['Episode'] = e
Group_DF['Scene'] = sc
Group_DF['Characters'] = ch
Group_DF.head(100)
c = 0
for index, group in Group_DF.iterrows():
  a = list(set(group[3].split(',')))
  contains_duplicates = any(a.count(element) > 1 for element in a)
  if contains_duplicates == True:
    c += 1

Interaction = pd.DataFrame(columns = ['Season','Episode','Scene','Character 1','Character 2'])
s = []
e = []
sc = []
c1 = []
c2 = []
for index,row in Group_DF.iterrows():
  season = row[0]
  episode = row[1]
  scene = row[2]
  a_list = list(set(row[3].split(',')))
  perm_list = itertools.permutations(a_list,2)
  perm_list = list(perm_list)
  for i in perm_list:
    c1.append(i[0])
    c2.append(i[1])
    s.append(season)
    e.append(episode)
    sc.append(scene)


Interaction['Season'] = s
Interaction['Episode'] = e
Interaction['Scene'] = sc
Interaction['Character 1'] = c1
Interaction['Character 2'] = c2

temp = Interaction.drop_duplicates()
Interaction = temp
Interaction['Character 1'] = Interaction['Character 1'].apply(lambda x: x.strip())
Interaction['Character 2'] = Interaction['Character 2'].apply(lambda x: x.strip())
Interaction = Interaction[Interaction['Character 1'] != Interaction['Character 2']]

In [ ]:
Interaction.to_excel('Interaction.xlsx')

In [ ]:
Interaction = pd.read_excel("New Interaction.xlsx")
Interaction = Interaction.drop(['Unnamed: 0', 'Unnamed: 0.1', 'Unnamed: 0.1.1'],axis=1)
Interaction.head(50)

,Season,Episode,Scene,Character 1,Character 2
0,1,1,Scene 1,The Boys,Kenny
1,1,1,Scene 1,The Boys,Kyle
2,1,1,Scene 1,The Boys,Cartman
3,1,1,Scene 1,The Boys,Ike
4,1,1,Scene 1,The Boys,Stan
5,1,1,Scene 1,Kenny,The Boys
6,1,1,Scene 1,Kenny,Kyle
7,1,1,Scene 1,Kenny,Cartman
8,1,1,Scene 1,Kenny,Ike
9,1,1,Scene 1,Kenny,Stan


In [ ]:
for index,row in Interaction.query('Season == 1').iterrows():
  print(index, row[3],row[4])

Streaming output truncated to the last 5000 lines.
541 Mr./Mrs. Garrison Mr. Hat
542 Kenny Cartman
543 Kenny Kyle
544 Kenny Stan
545 Cartman Kenny
546 Cartman Kyle
547 Cartman Stan
548 Kyle Kenny
549 Kyle Cartman
550 Kyle Stan
551 Stan Kenny
552 Stan Cartman
553 Stan Kyle
554 Johnson Mayor McDaniels
555 Johnson Ted
556 Mayor McDaniels Johnson
557 Mayor McDaniels Ted
558 Ted Johnson
559 Ted Mayor McDaniels
560 Stan Cartman
561 Stan Kyle
562 Cartman Stan
563 Cartman Kyle
564 Kyle Stan
565 Kyle Cartman
566 Mayor McDaniels Cows
567 Cows Mayor McDaniels
568 Mayor McDaniels Mr./Mrs. Garrison
569 Mr./Mrs. Garrison Mayor McDaniels
570 Bebe Mr./Mrs. Garrison
571 Bebe Clyde
572 Mr./Mrs. Garrison Bebe
573 Mr./Mrs. Garrison Clyde
574 Clyde Bebe
575 Clyde Mr./Mrs. Garrison
576 Bebe Mayor McDaniels
577 Mayor McDaniels Bebe
578 Bebe Mayor McDaniels
579 Bebe Mr./Mrs. Garrison
580 Mayor McDaniels Bebe
581 Mayor McDaniels Mr./Mrs. Garrison
582 Mr./Mrs. Garrison Bebe
583 Mr./Mrs. Garrison Mayor McDaniels

In [ ]:
df = pd.read_csv("All_Season_Script.csv")

In [ ]:
###Make an Episode Names List
Name = []
Seas = []
Epi = []
tag = 'https://en.wikipedia.org/wiki/List_of_South_Park_episodes'
page = requests.get(tag)
soup = BeautifulSoup(page.content,'html.parser')
for x in soup.find_all('td'):
  if ('"' in x.text.replace('\n','')) == True:
    Name.append(x.text.replace('\n','').replace('"',''))
Name = Name[:-15]

snum = 1
epinum = 1
for i in Name:
  if ((i == 'Terrance and Phillip in Not Without My Anus') or (i == 'Rainforest Shmainforest') or (i== "The Tooth Fairy's Tats 2000The Tooth Fairy's Tats") or (i == "It Hits the Fan") or (i == "Jared Has Aides") or (i == "Cancelled") or (i == "Good Times with Weapons") or (i == "Mr. Garrison's Fancy New Vagina") or (i == "The Return of Chef") or (i == "With Apologies to Jesse Jackson") or (i == "Tonsil Trouble") or (i == "The Ring") or (i == "Sexual Healing") or (i == "HumancentiPad") or (i == "Reverse Cowgirl") or (i == "Let Go, Let Gov") or (i == "Go Fund Yourself") or (i == "Stunning and Brave") or (i == "Member Berries") or (i == "White People Renovating Houses") or (i == "Dead Kids") or (i == "Mexican Joker")):
    snum += 1
    epinum = 1
  Seas.append(snum)
  Epi.append(epinum)
  epinum += 1

Episode = pd.DataFrame(columns=['Season','Episode','Episode Name'])
Episode['Season'] = Seas
Episode['Episode'] = Epi
Episode['Episode Name'] = Name
Episode['Episode Name'] = Episode['Episode Name'].str.replace("The Tooth Fairy's Tats 2000The Tooth Fairy's Tats","The Tooth Fairy's Tats 2000")
Episode['Episode Name'] = Episode['Episode Name'].str.replace("Cartman's Silly Hate Crime 2000Cartman's Silly Hate Crime","Cartman's Silly Hate Crime 2000")
Episode['Episode Name'] = Episode['Episode Name'].str.replace("Quintuplets 2000Contorting Quintuplets 2000Quintuplets","Quintuplets 2000")
Episode['Episode Name'] = Episode['Episode Name'].str.replace("PipGreat Expectations","Pip")

Episode.to_csv("Episode_Names.csv")
